# **Task 2**

Method a

**Naïve Bayes classifier**

In [131]:
# importing libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.multiclass import OneVsRestClassifier
from nltk.corpus import stopwords
from nltk import word_tokenize
from sklearn.metrics import accuracy_score, classification_report, multilabel_confusion_matrix
from sklearn.preprocessing import MultiLabelBinarizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import numpy as np
import nltk
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
import time

In [132]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [133]:
#function to get tag for words
def tagfunc(nltk_tag):
  if nltk_tag.startswith('J'):
    return wordnet.ADJ
  elif nltk_tag.startswith('V'):
    return wordnet.VERB
  elif nltk_tag.startswith('N'):
    return wordnet.NOUN
  elif nltk_tag.startswith('R'):
    return wordnet.ADV
  return None

# function  for lemmitization
def lemfunc(w):
  lemmatizer = WordNetLemmatizer()
  tag=nltk.pos_tag([w])[0]
  wordnet_tagged = [tag[0], tagfunc(tag[1])]
  desc=wordnet_tagged[1]
  if desc != None:
    return (lemmatizer.lemmatize(w,desc))
  else:
    return (lemmatizer.lemmatize(w))

# tokenization
def tokenfunc(i):
  t = nltk.word_tokenize(i)
  return t

#function for filtering
def filteringfunc(w, stop_words):
  ft = []
  for word in w:
    if word.isalpha() and word.lower() not in stop_words:
      ft.append(lemfunc(word))
  return ft




# stop words
custom_stopwords = {'movie', 'film', 'story', 'character', 'cinema'}
stop_words = set(stopwords.words('english'))
stop_words.update(custom_stopwords)

In [134]:
# preprocessing training dataset
cfp = './data/Training-dataset.csv'
filed = pd.read_csv(cfp)
data = filed['plot_synopsis'].tolist()
final = []
for i in data:
  final.append(filteringfunc(tokenfunc(i), stop_words))
filed['_processed'] = [' '.join(tokens) for tokens in final]
trainingX = filed['_processed']

In [135]:
# preprocessing validation dataset
cfp = './data/Task-2-validation-dataset.csv'
filedVal = pd.read_csv(cfp)
data = filedVal['plot_synopsis'].tolist()
final = []
for i in data:
  final.append(filteringfunc(tokenfunc(i), stop_words))
filedVal['_processed'] = [' '.join(tokens) for tokens in final]
validationX = filedVal['_processed']

In [136]:
# preprocessing test dataset
cfp = './data/Task-2-test-dataset2.csv'
filedTest = pd.read_csv(cfp)
data = filedTest['plot_synopsis'].tolist()
final = []
for i in data:
  final.append(filteringfunc(tokenfunc(i), stop_words))
filedTest['_processed'] = [' '.join(tokens) for tokens in final]
TestX = filedTest['_processed']


In [137]:

st = time.time()
ll = MultiLabelBinarizer()
ll.fit(filed[2:])
trainingY = ll.transform(filed[2:])
trainingY = filed.drop(['ID', 'title', 'plot_synopsis', '_processed'], axis=1)


In [138]:

v = CountVectorizer(max_df=0.75, min_df=0.01, ngram_range=(1, 3))
c = OneVsRestClassifier(MultinomialNB(alpha=0.0001))
#creating the pipeline
pipeline = make_pipeline(v, c)


In [139]:
pipeline.fit(trainingX, trainingY)
et=time.time()
print(et-st)

12.2162766456604


In [140]:
# Dimentionality of vectors
print("Dimensionality of vectors:", len(pipeline.named_steps['countvectorizer'].get_feature_names_out()))

Dimensionality of vectors: 4731


In [141]:
# Function to calculate predicted labels
def calculate_predicted_labels(pipeline, X):
    st = time.time()
    pred_lab = (pipeline.predict_proba(X) >= 0.84).astype(int)
    et = time.time()
    print(et-st)
    return pred_lab

# Function to create predicted labels dataframe
def create_predicted_labels_dataframe(predicted_labels, df, y):
    predicted_labels_df = pd.DataFrame(predicted_labels, columns = y.columns)
    predicted_labels_df['ID'] = df['ID'].values
    predicted_labels_df = predicted_labels_df[['ID'] + [col for col in predicted_labels_df.columns if col != 'ID']]
    return predicted_labels_df

# Function to save predicted labels to csv file
def save_predicted_labels_to_csv(predicted_labels_df, output_path):
    predicted_labels_df.to_csv(output_path, header=False, index=False)




In [142]:
# Applying on validation data
pred_lab = calculate_predicted_labels(pipeline, validationX)
print(pred_lab)
predicted_labels_df = create_predicted_labels_dataframe(pred_lab, filedVal, trainingY)
op = './data/10946159-Task2-method-a-validation.csv'
save_predicted_labels_to_csv(predicted_labels_df,op)

0.9269180297851562
[[0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 1 0 0]
 ...
 [0 1 0 ... 0 0 1]
 [1 0 1 ... 1 0 0]
 [1 0 0 ... 0 0 1]]


In [143]:
#!python3 '/content/task2_eval_script_student_version.py' '10946159-Task2-method-a-validation.csv' 'Task-2-validation-dataset.csv'

Class level: 
Class  1 precision: 0.2843 recall: 0.5086
Class  2 precision: 0.4244 recall: 0.5911
Class  3 precision: 0.4221 recall: 0.4422
Class  4 precision: 0.1184 recall: 0.3750
Class  5 precision: 0.7261 recall: 0.6936
Class  6 precision: 0.3390 recall: 0.5021
Class  7 precision: 0.5239 recall: 0.7172
Class  8 precision: 0.1279 recall: 0.3548
Class  9 precision: 0.5867 recall: 0.7167
----------------------------
Movie (document) level: 
Precision: 0.4995
Recall: 0.6282


In [144]:
# Applying on test data
predi_lab = calculate_predicted_labels(pipeline, TestX)
print(predi_lab)
predicted_labels_df = create_predicted_labels_dataframe(predi_lab, filedTest, trainingY)
op = './data/10946159-Task2-method-a.csv'
save_predicted_labels_to_csv(predicted_labels_df, op)

0.9017114639282227
[[0 0 0 ... 1 0 0]
 [0 0 0 ... 1 0 0]
 [1 1 1 ... 1 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [0 1 1 ... 0 0 0]
 [1 0 0 ... 0 0 0]]
